## Consolida todas as LDs em um único Dataframe:

In [1]:
# Importações necessárias
import pandas as pd
import os
from pathlib import Path
import numpy as np
import warnings

# Configuração para suprimir avisos
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

#### Cria uma coluna 'PLANNED DATE' com os valores mais atualizados da LD e Reordena a LD_final

In [2]:
def atualizar_planned_date_coalesce(df):
    """
    Atualiza a coluna 'PLANNED_DATE' do DataFrame com a data mais tarde entre
    'PLANNED_DATE', 'PLANNED_DATE_1' e 'PLANNED_DATE_2'.
    """
    # Converte as colunas para datetime, ignorando erros
    for col in ['PLANNED_DATE', 'PLANNED_DATE_1', 'PLANNED_DATE_2']:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')
        else:
            df[col] = pd.NaT

    # Calcula a data mais tarde entre as três colunas
    df['PLANNED_DATE'] = df[['PLANNED_DATE', 'PLANNED_DATE_1', 'PLANNED_DATE_2']].max(axis=1)

    return df

In [3]:
def clean_filename(filename):
    # Remove a extensão .xlsx
    filename = filename.replace('.xlsx', '').replace('.XLSX', '')
    # Pega apenas a parte do código do documento (antes do underscore)
    if '_' in filename:
        filename = filename.split('_')[0]
    return filename

# Caminho base para a pasta contendo os arquivos Excel
base_path = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\LDs\\TodasLDs\\'

# Dataframe para armazenar todos os resultados
df_LD_final = pd.DataFrame()

# Contadores para estatísticas
total_documentos = 0
documentos_com_comentarios = 0
documentos_com_Discipline = 0
cont_files = 0
arquivos_com_erro = []

# Iterar sobre todos os arquivos Excel na pasta
for filename in os.listdir(base_path):
    if not (filename.endswith('.xlsx') or filename.endswith('.XLSX')):
        continue
        
    file_path = os.path.join(base_path, filename)
    cont_files += 1
    print(f"Lendo arquivo: {file_path}")
    
    try:
        # Ler o arquivo Excel
        df = pd.read_excel(file_path, sheet_name="VDRL")
        
        # Pré-processamento dos dados
        df.columns = df.iloc[6]
        df = df.iloc[7:]
        
        # Tratar colunas duplicadas se existirem
        if df.columns.duplicated().any():
            # Criar um dicionário para contar ocorrências de cada nome de coluna
            col_counts = {}
            new_columns = []
            
            for col in df.columns:
                if col in col_counts:
                    col_counts[col] += 1
                    new_columns.append(f"{col}_{col_counts[col]}")
                else:
                    col_counts[col] = 0
                    new_columns.append(col)
            
            df.columns = new_columns
        
        
        # Renomear colunas padrão
        column_mapping = {
            "CLIENT DOCUMENT NUMBER": "CLIENT_DOCUMENT",
            "DOCUMENT TITLE": "DOCUMENT_TITLE",
            "PLANNED DATE": "PLANNED_DATE",
            "PLANNED DATE_1": "PLANNED_DATE_1",
            "PLANNED DATE_2": "PLANNED_DATE_2",
            "Discipline": "Discipline",
            "Comments": "Comments"
        }

        # Renomear apenas as colunas que existem
        for old_col, new_col in column_mapping.items():
            if old_col in df.columns:
                df.rename(columns={old_col: new_col}, inplace=True)
        
        # Atualizar a coluna PLANNED_DATE usando a função coalesce com a maior data
        df = atualizar_planned_date_coalesce(df)

        # Garantir que as colunas necessárias existem
        for col in ["CLIENT_DOCUMENT", "DOCUMENT_TITLE", "PLANNED_DATE", "Discipline", "Comments"]:
            if col not in df.columns:
                df[col] = None
        
        # Manter apenas as colunas necessárias
        colunas_para_manter = ["CLIENT_DOCUMENT", "DOCUMENT_TITLE", "PLANNED_DATE", "Discipline", "Comments"]
        df = df[colunas_para_manter]
        df.reset_index(drop=True, inplace=True)
        
        # Adicionar coluna com o nome completo do arquivo de origem
        df['Arquivo_Original'] = filename
        
        # Adicionar coluna com o nome do arquivo limpo
        df['LD'] = clean_filename(filename)
        
        # Atualizar estatísticas
        total_documentos += len(df)
        documentos_com_comentarios += df['Comments'].notna().sum()
        documentos_com_Discipline += df['Discipline'].notna().sum()
        
        # Concatenar com o DataFrame final
        df_LD_final = pd.concat([df_LD_final, df], axis=0, ignore_index=True)
        
    except Exception as e:
        print(f"Erro ao processar o arquivo {filename}: {str(e)}")
        arquivos_com_erro.append(filename)
        continue

# Resetar o índice do DataFrame final
df_LD_final.reset_index(drop=True, inplace=True)

# Função para limpar e converter para maiúsculas
def clean_text(x):
    if isinstance(x, str) and pd.notna(x):
        return x.strip().upper()
    return x

# Aplicar a função nas colunas
df_LD_final = df_LD_final.dropna(subset=['DOCUMENT_TITLE'])
df_LD_final['DOCUMENT_TITLE'] = df_LD_final['DOCUMENT_TITLE'].apply(clean_text)
df_LD_final['Discipline'] = df_LD_final['Discipline'].apply(clean_text)
df_LD_final['Comments'] = df_LD_final['Comments'].apply(clean_text)

# Função para substituir valores com "YES"
def convert_yes(value):
    if isinstance(value, str):
        value_upper = value.upper().strip()
        if 'YES' in value_upper or value_upper == 'Y' or value_upper == 'X' or value_upper == 'SIM':
            return 'YES'
    return ''

# Criar nova coluna 'Comments_new'
df_LD_final['Comments_new'] = df_LD_final['Comments'].apply(convert_yes)

# Atualizar estatísticas
total_documentos = len(df_LD_final)
documentos_com_comentarios = (df_LD_final['Comments_new'] == 'YES').sum()
documentos_com_Discipline = df_LD_final['Discipline'].notna().sum()

# Calcular estatísticas finais
if total_documentos > 0:
    porcentagem_com_comentarios = (documentos_com_comentarios / total_documentos) * 100
    porcentagem_com_Discipline = (documentos_com_Discipline / total_documentos) * 100
else:
    porcentagem_com_comentarios = 0
    porcentagem_com_Discipline = 0

# Criar DataFrame de estatísticas
df_estatisticas = pd.DataFrame({
    'Métrica': ['Total de Documentos', 'Documentos com YES', 'Documentos com Discipline',
                '% com Comentários', '% com Discipline', 'Arquivos com erro'],
    'Valor': [total_documentos, documentos_com_comentarios, documentos_com_Discipline,
              f"{porcentagem_com_comentarios:.2f}%", f"{porcentagem_com_Discipline:.2f}%", 
              len(arquivos_com_erro)]
})

# Salvar o DataFrame final e as estatísticas
output_file = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\LDs\\LD_comment_combined.xlsx'

with pd.ExcelWriter(output_file) as writer:
    df_LD_final.to_excel(writer, sheet_name='LD', index=False)
    df_estatisticas.to_excel(writer, sheet_name='Estatísticas', index=False)
    
    # Adicionar lista de arquivos com erro
    if arquivos_com_erro:
        df_erros = pd.DataFrame({'Arquivos com erro': arquivos_com_erro})
        df_erros.to_excel(writer, sheet_name='Erros', index=False)

print(f'Arquivo {output_file} salvo em: {Path(output_file).resolve()}')

# Imprimir estatísticas no console
print("\nMétricas:")
print(f"Número de arquivos processados: {cont_files}")
print(f"Total de Documentos: {total_documentos}")
print(f"Documentos com YES: {documentos_com_comentarios} ({porcentagem_com_comentarios:.2f}%)")
print(f"Documentos com Discipline: {documentos_com_Discipline} ({porcentagem_com_Discipline:.2f}%)")
print(f"Arquivos com erro: {len(arquivos_com_erro)}")
if arquivos_com_erro:
    print("Lista de arquivos com erro:")
    for arquivo in arquivos_com_erro:
        print(f"  - {arquivo}")

# Mostrar o DataFrame final
print("\nDataFrame final:")
print(f"Total de linhas no DataFrame final: {len(df_LD_final)}")

Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-1200-210-YA7-001_RISER HARD PIPE.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-1200-830-CT6-001_0_CHOKE VALVES.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-1200-911-EP8-096_0_FLOW METERING SYSTEM - rev1.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-1200-911-FK1-001_0_AXIAL_SDV.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-1200-911-HG8-001_PRESSURE VACUUM RELIEF VALVE.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-1200-911-HM9-001_0_PRESSURE VESSELS_TS.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-1200-911-HYE-001_NON_METALLIC_TANKS.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-1200-91

C:\Users\ELXY\AppData\Local\Temp\ipykernel_34944\2604130728.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce')


Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-1200-940-KD9-001_0_comments.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-1200-940-MKA-0001_0_BASKET FILTERS BRAZIL.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-120N-940-KD9-001_0_comments.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-1210-911-AP5-001_A_comments.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-1210-911-TYP-001_HPU_SUBSEA_comments.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-1223-940-CT6-001_0 - OIL SEPARATOR AND DEHYDRATOR.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-1223-940-HM9-001_B_comments.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-1223-940-YA7-001_Pressure Ve

In [4]:
# Define os caminhos dos arquivos
bd_path = 'C:\\Users\\ELXY\\Documents\\Codigos\\Python\\P84_85\\DadosBasicos\\'
orbis_path = 'C:\\Users\\ELXY\\Documents\\Codigos\\Python\\P84_85\\'

# Define o nome dos arquivos
bd_file = 'dados_basicos.xlsx'
orbis_file = 'orbis.xlsx'

df_db = pd.read_excel(os.path.join(os.path.dirname(bd_path), bd_file), sheet_name="LDxPKG")
df_orbis = pd.read_excel(os.path.join(os.path.dirname(orbis_path), orbis_file), sheet_name="Export")

# listar as linhas que tem PACKAGE duplicado
# df_db_duplicates = df_db[df_db.duplicated(subset=['PACKAGE'], keep=False)]
# Exibir as linhas duplicadas
#print("Linhas com PACKAGE duplicado:")
# display(df_db_duplicates)

# criar uma nova coluna chamada 'PKG_OK' no no df_db para verificar se o nome da coluna 'PACKAGE' existe no df_orbis na 'PKG DESCRIPTION'
df_db['PKG_OK'] = df_db['PACKAGE'].isin(df_orbis['PKG DESCRIPTION'])

#with pd.ExcelWriter("db.xlsx") as writer:
#    df_db.to_excel(writer, sheet_name='dados', index=False)

# Inserir as colunas 'Origem' e 'PACKAGE' no DataFrames df_LD_final mapeando as colunas LD do dataframe df_db com a coluna LD do dataframe df_LD_infal
df_LD_final['Origem'] = df_LD_final['LD'].map(df_db.set_index('LD')['Origem'])
df_LD_final['PACKAGE'] = df_LD_final['LD'].map(df_db.set_index('LD')['PACKAGE'])

In [5]:
# Remover do dataframe df_LD_final a coluna "Comments" não necessária
df_LD_final = df_LD_final.drop(columns=['Comments'])

# Ordenar as colunas do dataframe df_LD_final por CLIENT_DOCUMENT	DOCUMENT_TITLE	LD	Discipline Comments_new	PACKAGE Origem
df_LD_final = df_LD_final[['CLIENT_DOCUMENT', 'DOCUMENT_TITLE', 'LD', 'PLANNED_DATE', 'Discipline', 'Comments_new', 'PACKAGE', 'Origem']]

# Exibir o DataFrame final após remoção da coluna "Comments" e reordenação das colunas
#df_LD_final.info()

# Salvar resultados
with pd.ExcelWriter(output_file) as writer:
    df_LD_final.to_excel(writer, sheet_name='LD', index=False)

print(f'Arquivo {output_file} atualizado e salvo com as colunas "Origem" e "PACKAGE".')

Arquivo C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\LD_comment_combined.xlsx atualizado e salvo com as colunas "Origem" e "PACKAGE".
